In [100]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [8]:
train = pd.read_csv('last_train.csv')
test = pd.read_csv('last_test.csv')

In [9]:
train.drop('visitors_x',axis=1,inplace=True)
test.drop('visitors_x',axis=1,inplace=True)
train.rename(columns={'visitors_y':'visitors'},inplace=True)
test.rename(columns={'visitors_y':'visitors'},inplace=True)

In [10]:
data = pd.concat([train,test],axis=0)

In [11]:
data.columns

Index(['air_store_id', 'area', 'count_observations', 'date_int', 'dom',
       'dom_pattern_train_x', 'doq', 'dow', 'dow_pattern_2016', 'doy',
       'doy_pattern_2016', 'gen', 'genre_pop', 'holi', 'id', 'last_genre_pop',
       'last_month_comp', 'last_quarter_comp', 'last_quarter_gen_pop',
       'last_quarter_s_pop', 'lon_plus_lat', 'max_visitors', 'mean_visitors',
       'median_visitors', 'min_visitors', 'month', 'quarter', 'rs1_x', 'rs1_y',
       'rs2_x', 'rs2_y', 'rv1_x', 'rv1_y', 'rv2_x', 'rv2_y', 'store',
       'store_genre_pop', 'store_pop', 'store_pop_monthly',
       'total_reserv_dt_diff_mean', 'total_reserv_mean', 'total_reserv_sum',
       'var_max_lat', 'var_max_long', 'visit_date', 'visitors', 'year'],
      dtype='object')

In [5]:
data.shape

(282487, 48)

## holi features

In [25]:
mean = data.groupby(['year','month','dow','store'],as_index=False
                  )['visitors'].mean().rename(columns={'visitors':'dow_mean_monthly'})
median = data.groupby(['year','month','dow','store'],as_index=False
                     )['visitors'].median().rename(columns={'visitors':'dow_median_monthly'})
maxx = data.groupby(['year','month','dow','store'],as_index=False
                   )['visitors'].max().rename(columns={'visitors':'dow_max_monthly'})
minn = data.groupby(['year','month','dow','store'],as_index=False
                   )['visitors'].min().rename(columns={'visitors':'dow_min_monthly'})

In [26]:
mean.shape,median.shape,maxx.shape,minn.shape

((71505, 5), (71505, 5), (71505, 5), (71505, 5))

In [72]:
stats = pd.merge(mean,median,how='inner',on=['year','month','dow','store'])
stats = pd.merge(stats,maxx,how='inner',on=['year','month','dow','store'])
stats = pd.merge(stats,minn,how='inner',on=['year','month','dow','store'])

In [46]:
stats.shape

(71505, 8)

In [47]:
stats.head()

,year,month,dow,store,dow_mean_monthly,dow_median_monthly,dow_max_monthly,dow_min_monthly
0,0,0,0,3,33.666667,34.0,43,24
1,0,0,0,6,6.250000,6.0,11,2
2,0,0,0,8,10.000000,10.0,10,10
3,0,0,0,11,8.000000,8.0,8,8
4,0,0,0,12,24.000000,24.0,24,24


In [60]:
def get_median(data):
    result = [0,0,0,0,0,0,0]
    result[int(data.dow)] = data.dow_median_monthly
    return pd.Series(result,index=['dow0_median','dow1_median','dow2_median','dow3_median',
                                   'dow4_median','dow5_median','dow6_median'])
def get_mean(data):
    result = [0,0,0,0,0,0,0]
    result[int(data.dow)] = data.dow_mean_monthly
    return pd.Series(result,index=['dow0_mean','dow1_mean','dow2_mean','dow3_mean',
                                   'dow4_mean','dow5_mean','dow6_mean'])
def get_min(data):
    result = [0,0,0,0,0,0,0]
    result[int(data.dow)] = data.dow_min_monthly
    return pd.Series(result,index=['dow0_min','dow1_min','dow2_minn','dow3_min',
                                   'dow4_min','dow5_min','dow6_min'])
def get_max(data):
    result = [0,0,0,0,0,0,0]
    result[int(data.dow)] = data.dow_max_monthly
    return pd.Series(result,index=['dow0_max','dow1_max','dow2_max','dow3_max',
                                   'dow4_max','dow5_max','dow6_max'])

In [61]:
dow_median = stats.apply(get_median,axis=1)
dow_mean = stats.apply(get_mean,axis=1)
dow_min = stats.apply(get_min,axis=1)
dow_max = stats.apply(get_max,axis=1)

In [62]:
dow_median.head()

,dow0_median,dow1_median,dow2_median,dow3_median,dow4_median,dow5_median,dow6_median
0,34.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
stats = pd.concat([stats,dow_mean],axis=1)

In [74]:
stats.head()

,year,month,dow,store,dow_mean_monthly,dow_median_monthly,dow_max_monthly,dow_min_monthly,dow0_mean,dow1_mean,dow2_mean,dow3_mean,dow4_mean,dow5_mean,dow6_mean
0,0,0,0,3,33.666667,34.0,43,24,33.666667,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,6,6.250000,6.0,11,2,6.250000,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,8,10.000000,10.0,10,10,10.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,11,8.000000,8.0,8,8,8.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,12,24.000000,24.0,24,24,24.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
stats_ = stats.drop(['dow','dow_mean_monthly','dow_median_monthly',
                    'dow_max_monthly','dow_min_monthly'],axis=1)

In [76]:
stats_.shape

(71505, 10)

In [77]:
dow_stats = stats_.groupby(['year','month','store'],as_index=False
                          )['dow0_mean','dow1_mean','dow2_mean','dow3_mean',
                                   'dow4_mean','dow5_mean','dow6_mean',].sum()

In [78]:
dow_stats.shape

(10713, 10)

In [79]:
stats_.shape[0]/dow_stats.shape[0]

6.674600952114254

In [80]:
dow_stats.head()

,year,month,store,dow0_mean,dow1_mean,dow2_mean,dow3_mean,dow4_mean,dow5_mean,dow6_mean
0,0,0,3,33.666667,23.50,23.000000,32.50,30.500000,52.750000,74.250000
1,0,0,6,6.250000,14.25,8.000000,7.00,19.250000,19.750000,8.000000
2,0,0,8,10.000000,10.00,5.250000,7.50,14.333333,11.333333,8.500000
3,0,0,11,8.000000,16.00,12.250000,15.25,6.750000,6.000000,2.333333
4,0,0,12,24.000000,0.00,13.333333,15.00,15.666667,25.750000,24.250000


In [81]:
dow_stats = dow_stats.sort_values(['store','year','month'])

In [82]:
dow_stats.head(10)

,year,month,store,dow0_mean,dow1_mean,dow2_mean,dow3_mean,dow4_mean,dow5_mean,dow6_mean
1838,0,6,0,0.0,122.000000,13.000000,30.00,14.00,23.00,23.333333
2629,0,7,0,0.0,16.500000,19.600000,24.00,20.75,25.75,22.750000
3423,0,8,0,17.0,24.000000,20.666667,20.00,19.80,23.50,25.500000
4222,0,9,0,9.0,21.000000,20.500000,21.75,14.75,25.20,23.200000
5023,0,10,0,0.0,17.750000,20.500000,20.50,17.50,32.75,29.500000
5827,0,11,0,0.0,20.666667,23.333333,17.50,21.50,33.50,39.000000
6633,1,0,0,22.0,23.000000,13.666667,22.75,18.25,32.25,23.000000
7441,1,1,0,0.0,20.000000,20.666667,20.50,24.25,24.50,28.000000
8252,1,2,0,70.0,35.000000,32.750000,29.40,29.60,52.75,64.000000
9071,1,3,0,0.0,23.000000,27.750000,23.00,21.75,31.80,28.800000


In [83]:
dow_stats['store_shift'] = dow_stats.store.shift(1)
dow_stats.dow0_mean = dow_stats.dow0_mean.shift(1)
dow_stats.dow1_mean = dow_stats.dow1_mean.shift(1)
dow_stats.dow2_mean = dow_stats.dow2_mean.shift(1)
dow_stats.dow3_mean = dow_stats.dow3_mean.shift(1)
dow_stats.dow4_mean = dow_stats.dow4_mean.shift(1)
dow_stats.dow5_mean = dow_stats.dow5_mean.shift(1)
dow_stats.dow6_mean = dow_stats.dow6_mean.shift(1)

In [84]:
dow_stats.head()

,year,month,store,dow0_mean,dow1_mean,dow2_mean,dow3_mean,dow4_mean,dow5_mean,dow6_mean,store_shift
1838,0,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2629,0,7,0,0.0,122.0,13.000000,30.00,14.00,23.00,23.333333,0.0
3423,0,8,0,0.0,16.5,19.600000,24.00,20.75,25.75,22.750000,0.0
4222,0,9,0,17.0,24.0,20.666667,20.00,19.80,23.50,25.500000,0.0
5023,0,10,0,9.0,21.0,20.500000,21.75,14.75,25.20,23.200000,0.0


In [85]:
def fill_first(data,col):
    if data.store!=data.store_shift:
        return -1
    else:
        return data.loc[col]

In [86]:
dow_stats_ = dow_stats

In [89]:
for i in ['dow0_mean','dow1_mean','dow2_mean','dow3_mean','dow4_mean','dow5_mean','dow6_mean',]:
    def fill_first(data):
        if data.store!=data.store_shift:
            return -1
        else:
            return data.loc[i]
    dow_stats_[i] = dow_stats_.apply(fill_first,axis=1)

In [90]:
dow_stats_.head()

,year,month,store,dow0_mean,dow1_mean,dow2_mean,dow3_mean,dow4_mean,dow5_mean,dow6_mean,store_shift
1838,0.0,6.0,0.0,-1.0,-1.0,-1.000000,-1.00,-1.00,-1.00,-1.000000,NaN
2629,0.0,7.0,0.0,0.0,122.0,13.000000,30.00,14.00,23.00,23.333333,0.0
3423,0.0,8.0,0.0,0.0,16.5,19.600000,24.00,20.75,25.75,22.750000,0.0
4222,0.0,9.0,0.0,17.0,24.0,20.666667,20.00,19.80,23.50,25.500000,0.0
5023,0.0,10.0,0.0,9.0,21.0,20.500000,21.75,14.75,25.20,23.200000,0.0


In [91]:
dow_data = pd.merge(stats[['year','month','store','dow']],dow_stats_.drop('store_shift',axis=1),
                   how='left',on=['year','month','store'])

In [92]:
dow_data.shape

(71505, 11)

In [93]:
dow_data.dow.iloc[0]

0

In [94]:
def get_mean_data(data):
    if data.dow==0:
        return data.dow0_mean
    elif data.dow==1:
        return data.dow1_mean
    elif data.dow==2:
        return data.dow2_mean
    elif data.dow==3:
        return data.dow3_mean
    elif data.dow==4:
        return data.dow4_mean
    elif data.dow==5:
        return data.dow5_mean
    elif data.dow==6:
        return data.dow6_mean

In [95]:
dow_data['last_month_daily_mean'] = dow_data.apply(get_mean_data,axis=1)

In [96]:
dow_data.head()

,year,month,store,dow,dow0_mean,dow1_mean,dow2_mean,dow3_mean,dow4_mean,dow5_mean,dow6_mean,last_month_daily_mean
0,0,0,3,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0,0,6,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,0,0,8,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,0,0,11,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,0,0,12,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [97]:
dow_data.shape,data.shape

((71505, 12), (282487, 47))

In [98]:
data = pd.merge(data,dow_data[['year','month','store','dow','last_month_daily_mean']],how='inner',
               on = ['year','month','store','dow'])

In [107]:
data.year = data.year.astype(int)
data.month = data.month.astype(int)
data.store = data.store.astype(int)

## xgb test

In [110]:
data.columns

Index(['air_store_id', 'area', 'count_observations', 'date_int', 'dom',
       'dom_pattern_train_x', 'doq', 'dow', 'dow_pattern_2016', 'doy',
       'doy_pattern_2016', 'gen', 'genre_pop', 'holi', 'id', 'last_genre_pop',
       'last_month_comp', 'last_quarter_comp', 'last_quarter_gen_pop',
       'last_quarter_s_pop', 'lon_plus_lat', 'max_visitors', 'mean_visitors',
       'median_visitors', 'min_visitors', 'month', 'quarter', 'rs1_x', 'rs1_y',
       'rs2_x', 'rs2_y', 'rv1_x', 'rv1_y', 'rv2_x', 'rv2_y', 'store',
       'store_genre_pop', 'store_pop', 'store_pop_monthly',
       'total_reserv_dt_diff_mean', 'total_reserv_mean', 'total_reserv_sum',
       'var_max_lat', 'var_max_long', 'visit_date', 'visitors', 'year',
       'last_month_daily_mean'],
      dtype='object')

In [115]:
dtrain = xgb.DMatrix(data.drop(['id','visitors','air_store_id','visit_date','dom_pattern_train_x',
                                'dow_pattern_2016','last_month_daily_mean',
                               'last_genre_pop',
       'last_month_comp', 'last_quarter_comp', 'last_quarter_gen_pop'],axis=1),data.visitors.apply(np.log1p))

In [116]:
for i in np.arange(0.1,0.5,0.05):
    param = {
        'eta':i,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,30,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

0.1 0.295360666667
0.15 0.268213666667
0.2 0.265842
0.25 0.266170333333
0.3 0.268288666667
0.35 0.270475666667
0.4 0.273587333333
0.45 0.277174333333


In [119]:
set(data.year)

{0, 1}

## holi

In [120]:
holi_ = data[data.year==0].groupby(['month','holi','store'],as_index=False
                                  )['visitors'].median().rename(columns={'visitors':'holi_mean'})

In [122]:
holi_.shape

(11925, 4)

In [125]:
holi_yes = holi_[holi_.holi==1]
holi_no = holi_[holi_.holi==0]

In [126]:
holi_yes.shape,holi_no.shape

((5292, 4), (6633, 4))

In [127]:
holi_yes.head()

,month,holi,store,holi_mean
299,0,1,3,43.0
300,0,1,6,8.0
301,0,1,8,10.0
302,0,1,12,24.0
303,0,1,14,15.5


In [128]:
holi_no.rename(columns={'holi_mean':'nonholi_mean'},inplace=True)

C:\Users\Xufeng\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [129]:
holi_no.head()

,month,holi,store,nonholi_mean
0,0,0,3,34.0
1,0,0,6,13.0
2,0,0,8,9.0
3,0,0,11,8.0
4,0,0,12,20.0


In [130]:
data = pd.merge(data,holi_yes,how='left',on=['month','store'])
data = pd.merge(data,holi_no,how='left',on=['month','store'])

In [131]:
data.fillna(-1,inplace=True)

In [132]:
data.shape

(282487, 52)

In [133]:
data[['id','last_month_daily_mean','holi_mean','nonholi_mean']].to_csv('3features.csv',index=False)